In [ ]:
from datasets.stored_dataframe import CapgMyoDataModule, MyoArmbandDataModule, NinaProDataModule
from models import Classifier, OriginalModel
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torch
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchvision.transforms import Compose, ToTensor, Normalize
from models import LightningXGBClassifier
import os
from definitions import MODELS_FOLDER, ROOT_DIR
import warnings
from functools import partial
from tqdm import tqdm
import wandb
warnings.simplefilter(action='ignore', category=FutureWarning)
os.environ["CUDA_VISIBLE_DEVICES"]="2"

## Chinese Experiment

In [2]:
pl.seed_everything(42, workers=True)
k_folds = 10
lr_lambda=lambda epoch: 1 if epoch<16 else 0.1 if epoch<24 else 0.01
data_module_capgmyo = CapgMyoDataModule(
    batch_size=1000,
    k_folds=10
)
parameters = data_module_capgmyo.get_data_parameters()
lr_scheduler = torch.optim.lr_scheduler.LambdaLR
optimizer = torch.optim.RAdam
partial_classifier = partial(Classifier, optimizer=optimizer, lr_scheduler=lr_scheduler, optim_kwargs={'lr': 0.001, 'weight_decay': 0.0001}, monitor='val_accuracy', lr_lambda=lr_lambda)
for k in tqdm(range(k_folds)):
    classifier = partial_classifier(OriginalModel(parameters['num_classes'], parameters['input_width'], parameters['input_height']), fold=k)
    logger_capgmyo_chinese = WandbLogger(project="EMG Armband", name="Chinese CapgMyo", save_dir='wandb_logs')
    trainer_capgmyo_chinese = pl.Trainer(gpus=-1, max_epochs=28, logger=logger_capgmyo_chinese, accelerator="gpu")
    trainer_capgmyo_chinese.fit(model=classifier, datamodule=data_module_capgmyo)
    trainer_capgmyo_chinese.test(model=classifier, datamodule=data_module_capgmyo)
    wandb.finish()
    torch.save(classifier.model.state_dict(), os.path.join(MODELS_FOLDER, f'chinese_capgmyo_{k}_fold.pt'))
    data_module_capgmyo.next_fold()

NameError: name 'pl' is not defined

## CapgMyo Experiment

In [3]:
pl.seed_everything(42, workers=True)
k_folds = 10
data_module_capgmyo = CapgMyoDataModule(
    batch_size=1000,
    k_folds=10
)
parameters = data_module_capgmyo.get_data_parameters()
early_stop_callback = EarlyStopping(monitor='val_accuracy', patience=7, mode='max')
partial_classifier = partial(Classifier, optim_kwargs={'lr': 0.001, 'weight_decay': 0.0001}, monitor='val_accuracy', sched_kwargs={'patience': 4, 'mode': 'max'})
for k in tqdm(range(k_folds)):
    classifier = partial_classifier(OriginalModel(parameters['num_classes'], parameters['input_width'], parameters['input_height']), fold=k)
    logger_capgmyo = WandbLogger(project="EMG Armband", name="CapgMyo", save_dir='wandb_logs')
    trainer_capgmyo = pl.Trainer(gpus=-1, max_epochs=100, logger=logger_capgmyo, accelerator="gpu", callbacks=[early_stop_callback])
    trainer_capgmyo.fit(model=classifier, datamodule=data_module_capgmyo)
    trainer_capgmyo.test(model=classifier, datamodule=data_module_capgmyo)
    wandb.finish()
    torch.save(classifier.model.state_dict(), os.path.join(MODELS_FOLDER, f'capgmyo_{k}_fold.pt'))
    data_module_capgmyo.next_fold()

NameError: name 'pl' is not defined

## Ninapro Experiment

In [4]:
pl.seed_everything(42, workers=True)
k_folds = 10
data_module_ninapro = NinaProDataModule(
    batch_size=1000,
    k_folds=10
)
parameters = data_module_ninapro.get_data_parameters()
logger_ninapro = WandbLogger(project="EMG Armband", name="NinaPro")
early_stop_callback = EarlyStopping(monitor='val_accuracy', patience=7, mode='max')
partial_classifier = partial(Classifier, optim_kwargs={'lr': 0.001, 'weight_decay': 0.0001}, monitor='val_accuracy', sched_kwargs={'patience': 4, 'mode': 'max'})
for k in tqdm(range(k_folds)):
    classifier = partial_classifier(OriginalModel(parameters['num_classes'], parameters['input_width'], parameters['input_height']), fold=k)
    logger_ninapro = WandbLogger(project="EMG Armband", name="NinaPro", save_dir='wandb_logs')
    trainer_ninapro = pl.Trainer(gpus=-1, max_epochs=100, logger=logger_ninapro, accelerator="gpu", callbacks=[early_stop_callback])
    trainer_ninapro.fit(model=classifier, datamodule=data_module_ninapro)
    trainer_ninapro.test(model=classifier, datamodule=data_module_ninapro)
    wandb.finish()
    torch.save(classifier.model.state_dict(), os.path.join(MODELS_FOLDER, f'ninapro_{k}_fold.pt'))
    data_module_ninapro.next_fold()

NameError: name 'pl' is not defined

## MyoArmband Experiment

In [ ]:
pl.seed_everything(42, workers=True)
transform = Compose([
    ToTensor(),
    Normalize(0, 1)
])
data_module_myoarmband = MyoArmbandDataModule(
    batch_size=10000,
    num_workers=8,
    train_transforms=transform,
    val_transforms=transform,
    test_transforms=transform,
)
parameters = data_module_myoarmband.get_data_parameters()
early_stop_callback = EarlyStopping(monitor='val_accuracy', patience=7, mode='max')
partial_classifier = partial(Classifier, optim_kwargs={'lr': 0.001, 'weight_decay': 0.0001}, monitor='val_accuracy', sched_kwargs={'patience': 4, 'mode': 'max'})
for k in tqdm(range(k_folds)):
    classifier = partial_classifier(OriginalModel(parameters['num_classes'], parameters['input_width'], parameters['input_height']), fold=k)
    logger_myoarmband = WandbLogger(project="EMG Armband", name="MyoArmband", save_dir='wandb_logs')
    trainer_myoarmband = pl.Trainer(gpus=-1, max_epochs=100, logger=logger_myoarmband, accelerator="gpu", callbacks=[early_stop_callback])
    trainer_myoarmband.fit(model=classifier, datamodule=data_module_myoarmband)
    trainer_myoarmband.test(model=classifier, datamodule=data_module_myoarmband)
    wandb.finish()
    torch.save(classifier.model.state_dict(), os.path.join(MODELS_FOLDER, f'myoarmband_{k}_fold.pt'))
    data_module_myoarmband.next_fold()

## XGB Experiments

#### XGB CapgMyo

In [ ]:
data_module_xgb = CapgMyoDataModule(
    batch_size=1000,
    num_workers=3
)
parameters = data_module_xgb.get_data_parameters()
model = OriginalModel(parameters['num_classes'], parameters['input_width'], parameters['input_height'])
adjusted_model = model.load_state_dict(torch.load(os.path.join(MODELS_FOLDER, f'capgmyo_0_fold.pt')))
adjusted_model.model = adjusted_model.model[0:26]
partial_classifier = partial(LightningXGBClassifier, adjusted_model, data_module_xgb.num_classes, monitor='val_accuracy', tree_method='gpu_hist')
for k in tqdm(range(k_folds)):
    classifier = partial_classifier(OriginalModel(parameters['num_classes'], parameters['input_width'], parameters['input_height']), fold=k)
    logger_xgb = WandbLogger(project="EMG Armband", name="CapgMyoXGB", save_dir='wandb_logs')
    trainer_xgb = pl.Trainer(max_epochs=2, accelerator="cpu", logger=logger_xgb, num_sanity_val_steps=0)
    trainer_xgb.fit(model=classifier, datamodule=data_module_xgb)
    trainer_xgb.test(model=classifier, datamodule=data_module_xgb)
    wandb.finish()
    classifier.xgbmodel.save_model(os.path.join(MODELS_FOLDER, f'xgb_capgmyo_{k}_fold.json'))
    data_module_xgb.next_fold()

#### XGB Ninapro

In [ ]:
data_module_xgb = NinaProDataModule(
    batch_size=1000,
    num_workers=3
)
parameters = data_module_xgb.get_data_parameters()
model = OriginalModel(parameters['num_classes'], parameters['input_width'], parameters['input_height'])
adjusted_model = model.load_state_dict(torch.load(os.path.join(MODELS_FOLDER, f'ninapro_0_fold.pt')))
adjusted_model.model = adjusted_model.model[0:26]
partial_classifier = partial(LightningXGBClassifier, adjusted_model, data_module_xgb.num_classes, monitor='val_accuracy', tree_method='gpu_hist')
for k in tqdm(range(k_folds)):
    classifier = partial_classifier(OriginalModel(parameters['num_classes'], parameters['input_width'], parameters['input_height']), fold=k)
    logger_xgb = WandbLogger(project="EMG Armband", name="NinaproXGB", save_dir='wandb_logs')
    trainer_xgb = pl.Trainer(max_epochs=2, accelerator="cpu", logger=logger_xgb, num_sanity_val_steps=0)
    trainer_xgb.fit(model=classifier, datamodule=data_module_xgb)
    trainer_xgb.test(model=classifier, datamodule=data_module_xgb)
    wandb.finish()
    classifier.xgbmodel.save_model(os.path.join(MODELS_FOLDER, f'xgb_ninapro_{k}_fold.json'))
    data_module_xgb.next_fold()

#### XGB MyoArmband

In [ ]:
data_module_xgb = MyoArmbandDataModule(
    batch_size=1000,
    num_workers=3
)
parameters = data_module_xgb.get_data_parameters()
model = OriginalModel(parameters['num_classes'], parameters['input_width'], parameters['input_height'])
adjusted_model = model.load_state_dict(torch.load(os.path.join(MODELS_FOLDER, f'myoarmband_0_fold.pt')))
adjusted_model.model = adjusted_model.model[0:26]
partial_classifier = partial(LightningXGBClassifier, adjusted_model, data_module_xgb.num_classes, monitor='val_accuracy', tree_method='gpu_hist')
for k in tqdm(range(k_folds)):
    classifier = partial_classifier(OriginalModel(parameters['num_classes'], parameters['input_width'], parameters['input_height']), fold=k)
    logger_xgb = WandbLogger(project="EMG Armband", name="MyoArmbandXGB", save_dir='wandb_logs')
    trainer_xgb = pl.Trainer(max_epochs=2, accelerator="cpu", logger=logger_xgb, num_sanity_val_steps=0)
    trainer_xgb.fit(model=classifier, datamodule=data_module_xgb)
    trainer_xgb.test(model=classifier, datamodule=data_module_xgb)
    wandb.finish()
    classifier.xgbmodel.save_model(os.path.join(MODELS_FOLDER, f'xgb_myoarmband_{k}_fold.json'))
    data_module_xgb.next_fold()